In [1]:
!pip -q install rectools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.0/99.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 77.4 MB/s eta 0:00:00


In [3]:
import time
import logging
import requests
import pprint
from copy import deepcopy, copy
from typing import Dict, List, Tuple, Union, Callable, Any
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from IPython.display import display

In [9]:
import rectools
from rectools import Columns
from rectools.dataset import Interactions, Dataset, DenseFeatures
from rectools.model_selection import Splitter, TimeRangeSplitter
from rectools.models.base import ModelBase
from rectools.models import RandomModel, PopularModel
from rectools.metrics.base import MetricAtK
from rectools.metrics import (
    Precision,
    Recall,
    MAP,
    NDCG,
    Serendipity,
    MeanInvUserFreq,
    IntraListDiversity,
    PairwiseHammingDistanceCalculator,
    calc_metrics,
)

In [5]:
url = 'https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip'

In [6]:
req = requests.get(url, stream=True)

with open('kion.zip', 'wb') as fd:
    total_size_in_bytes = int(req.headers.get('Content-Length', 0))
    progress_bar = tqdm(desc='kion dataset download', total=total_size_in_bytes, unit='iB', unit_scale=True)
    for chunk in req.iter_content(chunk_size=2 ** 20):
        progress_bar.update(len(chunk))
        fd.write(chunk)

kion dataset download:   0%|          | 0.00/78.8M [00:00<?, ?iB/s]

In [7]:
import zipfile as zf

files = zf.ZipFile('kion.zip','r')
files.extractall()
files.close()

In [54]:
interactions = pd.read_csv('data_original/interactions.csv', parse_dates=["last_watch_dt"])

interactions.rename(
    columns={
        'last_watch_dt': Columns.Datetime,
        'total_dur': Columns.Weight
    },
    inplace=True)

In [11]:
users = pd.read_csv('data_original/users.csv')
items = pd.read_csv('data_original/items.csv')

In [14]:
interactions.head(5)

,user_id,item_id,datetime,weight,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0


In [13]:
users.head(5)

,user_id,age,income,sex,kids_flg
0,973171,age_25_34,income_60_90,М,1
1,962099,age_18_24,income_20_40,М,0
2,1047345,age_45_54,income_40_60,Ж,0
3,721985,age_45_54,income_20_40,Ж,0
4,704055,age_35_44,income_60_90,Ж,0


In [15]:
items.head(5)

,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."


Возьмем метрики: классификационные (Precision, Recall), метрики ранжирования (MAP, NDCG), специальные (Serendipity, Novelty)

In [68]:
metrics = {
          "precision": Precision,
          "recall": Recall,
          "MAP": MAP,
          "NDCG": NDCG,
          "serendipity": Serendipity,
          "novelty": MeanInvUserFreq
    }

k = [1, 5, 10]

In [69]:
def compute_metrics(metrics, K):
    metrics_at_k = {}

    for key, value in metrics.items():
        kwargs = {}
        if isinstance(value, tuple):
            kwargs.update(**value[1])
            value = value[0]

        metrics_at_k.update({f"{key}@{k}": value(k=k, **kwargs) for k in K})

    return metrics_at_k

In [70]:
metrics_at_k = compute_metrics(metrics, k)

In [71]:
metrics_at_k

{'precision@1': Precision(k=1),
 'precision@5': Precision(k=5),
 'precision@10': Precision(k=10),
 'recall@1': Recall(k=1),
 'recall@5': Recall(k=5),
 'recall@10': Recall(k=10),
 'MAP@1': MAP(k=1, divide_by_k=False),
 'MAP@5': MAP(k=5, divide_by_k=False),
 'MAP@10': MAP(k=10, divide_by_k=False),
 'NDCG@1': NDCG(k=1, log_base=2),
 'NDCG@5': NDCG(k=5, log_base=2),
 'NDCG@10': NDCG(k=10, log_base=2),
 'serendipity@1': Serendipity(k=1),
 'serendipity@5': Serendipity(k=5),
 'serendipity@10': Serendipity(k=10),
 'novelty@1': MeanInvUserFreq(k=1),
 'novelty@5': MeanInvUserFreq(k=5),
 'novelty@10': MeanInvUserFreq(k=10)}

In [72]:
def computation_metrics(models, metrics, splitter, interactions, k):

    results = []
    interactions_data = Interactions(interactions)
    fold_iterator = splitter.split(interactions_data, collect_fold_stats=True)

    for train_ids, test_ids, fold_info in fold_iterator:

        df_train = interactions_data.df.iloc[train_ids]
        df_test = interactions_data.df.iloc[test_ids][Columns.UserItem]

        dataset = Dataset.construct(df_train)
        test_users = np.unique(df_test[Columns.User])

        catalog = df_train[Columns.Item].unique()

        for key, value in models.items():

            model = deepcopy(value)

            start = time.time()
            model.fit(dataset)
            stop = time.time()

            reco = model.recommend(users=test_users, dataset=dataset, k=k, filter_viewed=True)

            metric_values = calc_metrics(
                metrics,
                reco=reco,
                interactions=df_test,
                prev_interactions=df_train,
                catalog=catalog,
            )

            times = {"model": key, "time": stop - start}
            times.update(metric_values)
            results.append(times)

    results_df = pd.DataFrame(results)

    return results_df

Сплиттер: 3 фолда для кросс-валидации по неделе, исключение холодных юзеров и айтемов и просмотренных айтемов

In [73]:
splitter = TimeRangeSplitter(test_size="7D", n_splits=3, filter_cold_users=True, filter_cold_items=True, filter_already_seen=True)

In [74]:
models = {"RandomModel": RandomModel(), "PopularModel": PopularModel()}

results = computation_metrics(models, metrics_at_k, splitter, interactions, 10)

In [75]:
pd.DataFrame(results).groupby("model").mean()

,time,precision@1,recall@1,precision@5,recall@5,precision@10,recall@10,NDCG@1,NDCG@5,NDCG@10,MAP@1,MAP@5,MAP@10,novelty@1,novelty@5,novelty@10,serendipity@1,serendipity@5,serendipity@10
model,,,,,,,,,,,,,,,,,,,
PopularModel,1.865071,0.076432,0.042720,0.052402,0.137413,0.033903,0.173492,0.076432,0.057932,0.043084,0.042720,0.078295,0.084109,2.377055,3.066979,3.713390,0.000002,0.000003,0.000002
RandomModel,0.000046,0.000180,0.000066,0.000170,0.000318,0.000166,0.000601,0.000180,0.000170,0.000167,0.000066,0.000148,0.000185,15.613740,15.614079,15.613485,0.000006,0.000006,0.000006


In [92]:
def visualized(model, dataset, users, features, items):

  if model.is_fitted == False:
        raise Exception('model is not fitted')

  recommendations = model.recommend(users, dataset, 3, True)
  interactions_data = dataset.interactions.df
  history = interactions_data[interactions_data.user_id.isin(users)]

  for user in users:

    history_for_user = history[history.user_id.isin([user])]
    hist = items.join(history_for_user.set_index('item_id'), on='item_id')[["user_id"] + features]
    hist_for_id = hist.loc[hist['user_id'].isin([user])]
    print(f"Пользователь с id {user} уже посмотрел:")
    print(hist_for_id)
    print('')

    recommendations_for_id = model.recommend(users, dataset, 3, True)
    rec = pd.merge(recommendations_for_id, items)[["user_id"] + features]
    rec_for_id = rec.loc[rec['user_id'].isin([user])]

    print(f"Пользователю с id {user} можно порекомендовать:")
    print(rec_for_id)
    print('')
    print('')

In [93]:
model = RandomModel(random_state=32)
dataset = Dataset.construct(interactions)
model.fit(dataset)
users = np.array([666262, 672861, 955527])

In [94]:
visualized(model, dataset, users, ["title"], items)

Пользователь с id 666262 уже посмотрел:
        user_id                 title
11230  666262.0  Дом ночных призраков

Пользователю с id 666262 можно порекомендовать:
   user_id                                         title
0   666262                           Возвращение Будулая
1   666262  Новые приключения Аладдина (жестовым языком)
2   666262                             Пропавшая грамота


Пользователь с id 672861 уже посмотрел:
        user_id                          title
11182  672861.0                 В ритме сердца
13578  672861.0  Медвежонок Винни и его друзья

Пользователю с id 672861 можно порекомендовать:
   user_id                     title
3   672861          Женщина в беде 3
4   672861  Гордость и предубеждение
5   672861                Болванчики


Пользователь с id 955527 уже посмотрел:
       user_id        title
8909  955527.0  Признание 5

Пользователю с id 955527 можно порекомендовать:
   user_id                      title
6   955527               Солдат Джейн
7   